In [11]:
%pip install requests
%pip install tabulate
%pip install future

# Required for plotting:
%pip install matplotlib

!sudo apt update
!sudo apt install openjdk-11-jdk


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 https://download.docker.com/linux/ubuntu focal InRelease [57.7 kB]       
Get:3 https://cli.github.com/packages stable InRelease [3917 B]                
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:7 https://download.docker.com/linux/ubuntu focal/stable amd64 Packages [57.5 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4344 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-up

In [12]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error, matthews_corrcoef
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv





import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
from autogluon.tabular import TabularDataset, TabularPredictor

import h2o
from h2o.automl import H2OAutoML

import gc

from xgboost import plot_importance
import wandb
from fastai.callback.wandb import *
from wandb.integration.lightgbm import wandb_callback, log_summary
#from wandb.xgboost import wandb_callback



#from IPython.display import FileLink

In [13]:
path = Path('playground-series-s4e8/')
path

Path('playground-series-s4e8')

In [14]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [15]:
# Make a copy of the training dataframe to avoid modifying the original
train_df_with_nan = train_df.copy()

# Add a NaN value to a random row in the 'stem-height' column
random_index = np.random.choice(train_df_with_nan.index)
train_df_with_nan.loc[random_index, 'stem-height'] = np.nan

# Verify the NaN was added
print(train_df_with_nan['stem-height'].isna().sum())

1


In [16]:
cont_names,cat_names = cont_cat_split(train_df_with_nan, dep_var='class')
len(cat_names),len(cont_names)

(17, 3)

In [17]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df_with_nan))

In [18]:
to = TabularPandas(train_df_with_nan, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='class',
                   y_block=CategoryBlock(),
                   splits=splits)

In [19]:
dls = to.dataloaders(bs=64)
#dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test_df)

In [20]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

# H2O

In [21]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpuykcezpz
  JVM stdout: /tmp/tmpuykcezpz/h2o_zeus_started_from_python.out
  JVM stderr: /tmp/tmpuykcezpz/h2o_zeus_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,H2O_from_python_zeus_dew1zc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.836 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [22]:
train_data = h2o.H2OFrame(train_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [23]:
train_data['class'] = train_data['class'].asfactor()

In [24]:
aml = H2OAutoML(max_runtime_secs=3600,seed=42)
#aml = H2OAutoML(max_runtime_secs=900,seed=42)
aml.train(y='class', training_frame=train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,blending
Number of base models (used / total),6/12
# GBM base models (used / total),1/5
# XGBoost base models (used / total),3/3
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),1/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0


In [25]:
leaderboard = aml.leaderboard
print(leaderboard)
best_model = aml.leader
print(best_model)

model_id                                                     auc    logloss     aucpr    mean_per_class_error       rmse         mse
StackedEnsemble_AllModels_2_AutoML_1_20240811_60211     0.997176  0.0363553  0.997586              0.00762358  0.0839055  0.00704013
StackedEnsemble_AllModels_1_AutoML_1_20240811_60211     0.997162  0.0364036  0.997541              0.00766584  0.0839201  0.00704258
StackedEnsemble_BestOfFamily_3_AutoML_1_20240811_60211  0.997152  0.0365004  0.997555              0.00766581  0.0840686  0.00706754
StackedEnsemble_BestOfFamily_2_AutoML_1_20240811_60211  0.997149  0.0365122  0.997547              0.00768211  0.0840598  0.00706605
StackedEnsemble_BestOfFamily_1_AutoML_1_20240811_60211  0.997149  0.0365282  0.997552              0.00767416  0.0840947  0.00707191
StackedEnsemble_BestOfFamily_4_AutoML_1_20240811_60211  0.99714   0.0365049  0.997526              0.00764715  0.0840121  0.00705804
GBM_1_AutoML_1_20240811_60211                           0.997067  0.0

In [26]:
df_test = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [27]:
predictions = best_model.predict(df_test)
predictions_df = predictions.as_data_frame()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [28]:
predictions_df

,predict,e,p
0,e,0.997204,0.002796
1,p,0.001985,0.998015
2,p,0.006181,0.993819
3,p,0.004875,0.995125
4,e,0.998023,0.001977
...,...,...,...
2077959,p,0.002077,0.997923
2077960,p,0.006775,0.993225
2077961,p,0.004918,0.995082
2077962,e,0.996968,0.003032


In [29]:
!ls

AutogluonModels  playground-series-s4e8		     submission.csv
lgbm_model.pkl	 playground-series-s4e8.zip	     wandb
main.py		 poisonous_mushrooms_classification  xgb_model.json
models		 requirements.txt


In [30]:
!rm submission.csv

In [31]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] =(predictions_df['predict'].values)
submit.to_csv('submission.csv', index=False)

In [32]:
sub = pd.read_csv('submission.csv')
print(sub)

              id class
0        3116945     e
1        3116946     p
2        3116947     p
3        3116948     p
4        3116949     e
...          ...   ...
2077959  5194904     p
2077960  5194905     p
2077961  5194906     p
2077962  5194907     e
2077963  5194908     e

[2077964 rows x 2 columns]


In [30]:
!rm submission.csv

In [ ]:
#df_sub['class'] =(predictions_df['predict'].values)
#df_sub.to_csv('submission.csv', index=False)

In [33]:
!kaggle competitions submit -c playground-series-s4e8 -f submission.csv -m "[H2O] Silver Rubanza, H2O max_runtime_secs=3600, - lightning  "

100%|██████████████████████████████████████| 19.8M/19.8M [00:00<00:00, 46.5MB/s]
Successfully submitted to Binary Prediction of Poisonous Mushrooms